In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
customer = "Asambeauty"
audience_id = "6687daf21767991e3bdfb414"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [4]:
audiences = call_api_with_accountId(
    f"{url}/audiences/query",
    account_id,
    {"id":audience_id},
    logging
)
if len(audiences) != 1:
    raise Exception(f"more or less than one source was found: {audiences}")
list_treatments = audiences[0]["config"]["treatments"]
print(f"treatments = {len(list_treatments)}")
list_treatments

In [5]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":list_treatments},
    logging
)
treatments = pd.json_normalize(treatments)
treatments

In [6]:
treatments["source"].value_counts()

In [7]:
treatments["properties.status"].value_counts()

In [8]:
treatments["relates_to.campaign.name"].value_counts()

In [9]:
treatments[treatments["properties.status"] == "active"]["relates_to.campaign.name"].value_counts()

In [10]:
treatments.columns

In [11]:
if 'relates_to.adset.name' in treatments.columns:
    vc = treatments.groupby(by=["relates_to.campaign.name","id"])['relates_to.adset.name'].value_counts()
else:
    vc = treatments.groupby(by=["relates_to.campaign.name","id"])['relates_to.assetGroup.name'].value_counts()
vc.reset_index()